## YOLO

### PyTorch 기반 물체인식 모델
- CNN, rCNN(Regions with CNN)

#### YOLOv5 이상 설치
```shell
> pip install ultralytics
```

In [1]:
# YOLO설치
!pip install ultralytics


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 콘솔에서 예측

In [2]:
# 콘솔에서 예측
# yolo11n.pt - pretrained YOLO model
!yolo predict model=yolo11n.pt source='./bus.jpg'


^C


Ultralytics 8.3.109 🚀 Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 c:\Source\iot-dataanalysis-2025\day08\bus.jpg: 640x480 4 persons, 1 bus, 37.8ms
Speed: 5.6ms preprocess, 37.8ms inference, 78.6ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\Source\iot-dataanalysis-2025\runs\detect\predict3
💡 Learn more at https://docs.ultralytics.com/modes/predict


### 파이썬으로 예측

In [3]:
# YOLO 모듈 로드
from ultralytics import YOLO

In [4]:
# YOLO 클래스가 들어오는 모델의 버전에 따라 알아서 예측모델 객체 생성
model = YOLO('./yolo11n.pt')

#### coco8.yaml
- https://github.com/ultralytics/assets/releases/download/v0.0.0/coco8.zip
- 위 내용대로 훈련을 시킨 결과 -> yolo11n.pt

In [ ]:
# coco8.yaml - YOLO 훈련에 사용할 데이터셋 정의파일
train_results = model.train(
    data='./coco8.yaml',
    epochs=100,
    imgsz=640,
    device = 'cuda:0'
)

#### 이미지 예측

In [8]:
result = model('./0000001.jpg')


image 1/1 c:\Source\iot-dataanalysis-2025\day08\0000001.jpg: 480x640 1 cat, 33.6ms
Speed: 1.3ms preprocess, 33.6ms inference, 71.1ms postprocess per image at shape (1, 3, 480, 640)


In [6]:
# matplotlib 모듈 로드
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [9]:
img = result[0].plot()
img_pil = Image.fromarray(img[...,::-1])
img_pil.save('./predict_result.jpg')

#### OpenCV
- Opensource Computer Vision 약자. 실시간 컴퓨터 비전(시각처리)을 목적으로 프로그래밍 라이브러리
- 인텔에서 2000년에 C, C++ 사용하기 위해서 개발
- 파이썬에서 사용할 수 있게 래핑
```shell
> pip install opencv-python
```

In [ ]:
# OpenCV 설치
!pip install opencv-python

In [10]:
import cv2
cv2.__version__

'4.11.0'

In [11]:
img2 = cv2.imread('./predict_result.jpg')
img2.shape  # (464, 640, 3) -> (height, width, channal)

(464, 640, 3)

In [12]:
# 윈도우 창 오픈
cv2.imshow('결과', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()


#### YOLO 예측

In [13]:
img = cv2.imread('./0000002.jpg')
resized_img = cv2.resize(img, (640, 400))

result = model(resized_img)
plots = result[0].plot()

cv2.imshow('predict_result', plots)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 416x640 1 bottle, 1 cup, 1 bowl, 1 tv, 1 mouse, 1 book, 50.5ms
Speed: 2.5ms preprocess, 50.5ms inference, 2.4ms postprocess per image at shape (1, 3, 416, 640)


YOLO 실시간 예측

In [60]:
video_path = './sample01.mp4'
output_path = './sample01_output.mp4'
count_path = './sample01_count.mp4'

In [36]:
# 시간 모듈 로드
import time

In [41]:
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)        # 동영상 FPS(Frame Per Second)
frame_time = 1.0 / fps                 # 초단위로 변환
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))    # 1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 720

# VideoWriter 객체 생성(동영상 화면에 그림, 글자를 그리기 위한 객체)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    start_time = time.time() # 시작시간
    ret, frame = cap.read()
    if not ret: break

    # 객체 탐지
    results = model(frame)
    # 탐지 결과를 그리기
    for result in results:
        detect_frame = result.plot()
    # 결과프레임을 파일로 저장
    out.write(detect_frame)
    # 결과 표시
    cv2.imshow('YOLO Object Detection', detect_frame)
    cv2.imshow('Video play', frame)

    # 프레임간 실제 지연시간 계산
    elapsed_time = time.time() - start_time
    delay = max(int((frame_time - elapsed_time)*1000),-1)

    cv2.imshow('Video play', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # q 버튼을 누르면
        break

cap.release() # 비디오를 해제
cv2.destroyAllWindows()



0: 384x640 1 train, 127.7ms
Speed: 10.7ms preprocess, 127.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 train, 12.3ms
Speed: 2.0ms preprocess, 12.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 14.5ms
Speed: 1.7ms preprocess, 14.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 12.6ms
Speed: 2.1ms preprocess, 12.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 train, 11.6ms
Speed: 1.8ms preprocess, 11.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 12.1ms
Speed: 1.5ms preprocess, 12.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 train, 11.8ms
Speed: 1.3ms preprocess, 11.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 11.8ms
Speed: 1.6ms preprocess, 11.8ms inference, 1.6ms postprocess per image at

#### Car Counting
- 지정된 라인 아래로 내려오는 자동차 개수 카운팅

In [50]:
import cv2
from ultralytics import solutions

- shapely 설치
```shell
> pip install shapely==2.0.1
```

In [52]:
!pip install shapely==2.0.1


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
# lap 설치
!pip install lap

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.5 MB 1.8 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 5.1 MB/s eta 0:00:01
   ------------------------------- -------- 1.2/1.5 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 7.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
import cv2
from ultralytics.solutions import ObjectCounter

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), 'Error reading video file'  # 파일 열리지 않으면 경고처리

region_points = [(20, 400), (1080, 400)]  # 라인수
fps = cap.get(cv2.CAP_PROP_FPS)  # 동영상 FPS(Frame Per Second)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 720
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(count_path, fourcc, fps, (width, height))

counter = ObjectCounter(
    show=True,                # 처리중 화면에 디스플레이
    region=region_points,     # 라인위치
    model='yolo11n.pt',      # YOLO 모델
    # classes=[0, 2],
    # tracker='botsort.yaml',
)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    results = counter(frame)
    out.write(results.plot_im)  # 여기 차이

cap.release()
out.release()
cv2.destroyAllWindows()


Ultralytics Solutions:  {'region': [(20, 400), (1080, 400)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'show': True, 'model': 'yolo11n.pt'}

0: 384x640 1 train, 12.9ms
Speed: 1.8ms preprocess, 12.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(classwise_count={'train': {'IN': 0, 'OUT': 0}}, total_tracks=1)

0: 384x640 1 train, 14.3ms
Speed: 2.2ms preprocess, 14.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(classwise_count={'train': {'IN': 0, 'OUT': 0}}, total_tracks=1)

0: 384x640 1 train, 11.2ms
Speed: 1.4ms preprocess, 11.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(classwise_count={'train': {'IN': 0, 'OUT': 0}}, total_tracks=1)

0: 384x640 1 train, 12.6ms
Speed: 1.3ms preprocess, 12.6ms inference, 1.6ms postprocess